In [5]:
from langchain_community.document_loaders import TextLoader #this takes the raw text of the books description and convert it into a format that langching can work with
from langchain_text_splitters import CharacterTextSplitter #split the whole doc containing all the description into meaningful chunks 
from langchain_openai import OpenAIEmbeddings #API to call the model
from langchain_chroma import Chroma #use to store the info's into the vector database 
from langchain.embeddings import HuggingFaceEmbeddings


# API LOAD

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
import pandas as pd
books=pd.read_csv("books_cleaned.csv")


In [8]:
books["tagged_description"]
#appended the ISBN as an identifier

0       9780002005883 A NOVEL THAT READERS and critics...
1       9780002261982 A new 'Christie for Christmas' -...
2       9780006178736 A memorable, mesmerizing heroine...
3       9780006280897 Lewis' work on the nature of lov...
4       9780006280934 "In The Problem of Pain, C.S. Le...
                              ...                        
5192    9788172235222 On A Train Journey Home To North...
5193    9788173031014 This book tells the tale of a ma...
5194    9788179921623 Wisdom to Create a Life of Passi...
5195    9788185300535 This collection of the timeless ...
5196    9789027712059 Since the three volume edition o...
Name: tagged_description, Length: 5197, dtype: object

In [9]:
#split off that ISBN from the front of the description and use that as the thing to filter

books["tagged_description"].to_csv("tagged_description.txt", lineterminator="\n", index=False, header=False)

In [10]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
document = text_splitter.split_documents(raw_documents)

#this step helps in better text processing, searching, embeddings, or recommendations.

Created a chunk of size 1170, which is longer than the specified 0
Created a chunk of size 1216, which is longer than the specified 0
Created a chunk of size 375, which is longer than the specified 0


Created a chunk of size 311, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 484, which is longer than the specified 0
Created a chunk of size 962, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 845, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1090, which is longer than the specified 0
Created a chunk of size 1191, which is longer than the specified 0
Created a chunk of size 306, which is longer than the specified 0
Created a chunk of size 270, which is longer than the specified 0
Created a chunk of size 213, which is longer than the specified 0
Created a chunk of size 216, which is longer than the specified 0
Created 

`CharacterTextSplitter` is a text-splitting tool that breaks text into smaller chunks based on characters.\
`Chunk_size=0`: Ensures that the text is only split at separators and does not try to split based on chunk size.\
If `chunk_size > 0`, the splitter tries to break the text based on the closest separator near the chunk size.\
If `chunk_size=0`, it ignores chunk size and prioritizes the separator (\n), ensuring clean splits at every new line.\
`chunk_overlap=0`: Ensures no overlapping between chunks.\
`separator="\n"`:\
Specifies that text should be split at new lines (\n).\
This makes sense since each book description is likely on a separate line in tagged_description.txt.

In [11]:
document[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

## Storing into Vector Databse


In [12]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [15]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db_books = Chroma.from_documents(document, embedding=embedding_model)


In [16]:
query = "a book to teach childern about nature"
docs= db_books.similarity_search(query, k=10)
docs

[Document(id='f885d12d-b093-43fe-9a4f-5cdb9f98ce0d', metadata={'source': 'tagged_description.txt'}, page_content='"9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience."'),
 Document(id='f33b83fc-6df9-4130-874d-c871ffbd2b0a', metadata={'source': 'tagged_description.txt'}, page_content='"9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child\'s drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."'),
 Document(id='ce64e7ea-421e-4caa-86bd-c9e8c34b9aa9', metadata={'sourc

In [17]:
books[books["isbn13"]== int(docs[0].page_content.split()[0].strip().replace('"',""))]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [18]:
def retrive_semantic_recommendations(query: str, top_k: int = 10,) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=50)  # Fetch similar books
    books_list = []

    for i in range(len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)].head(top_k)

In [19]:
retrive_semantic_recommendations("A book to teach childer about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
59,9780007151240,0007151241,The Family Way,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,The Family Way,9780007151240 It should be the most natural th...
324,9780060959036,0060959037,Prodigal Summer,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
383,9780061144899,0061144894,When the Heart Waits,Sue Monk Kidd,Religion,http://books.google.com/books/content?id=JlP91...,From the Bestselling Author of The Secret Life...,2006.0,4.17,240.0,2141.0,When the Heart Waits: Spiritual Direction for ...,9780061144899 From the Bestselling Author of T...
404,9780064402453,0064402452,Racso and the Rats of NIMH,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit..."
406,9780064403870,0064403874,"R-T, Margaret, and the Rats of NIMH",Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=WTHHH...,"When Margaret and her younger brother, Artie, ...",1991.0,3.52,272.0,631.0,"R-T, Margaret, and the Rats of NIMH",9780064403870 When Margaret and her younger br...
407,9780064404419,0064404412,The Rainbow People,Laurence Yep,Juvenile Fiction,http://books.google.com/books/content?id=5AHwq...,"""Culled from 69 stories collected in a [1930s]...",1992.0,3.75,208.0,202.0,The Rainbow People,"9780064404419 ""Culled from 69 stories collecte..."
416,9780064406925,006440692X,Winter on the Farm,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=IvlKH...,The Little House books tell the story of a lit...,1997.0,4.13,32.0,400.0,Winter on the Farm,9780064406925 The Little House books tell the ...
427,9780064434874,0064434877,Christmas in the Big Woods,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=FT1Yp...,"Long ago, a little girl named Laura Ingalls li...",1997.0,4.19,32.0,2062.0,Christmas in the Big Woods,"9780064434874 Long ago, a little girl named La..."
429,9780064434980,0064434982,The Deer in the Wood,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=V7YDW...,Even the youngest child can enjoy a special ad...,1999.0,4.17,32.0,302.0,The Deer in the Wood,9780064434980 Even the youngest child can enjo...
436,9780064471961,0064471969,Shade's Children (rack),Garth Nix,Juvenile Fiction,http://books.google.com/books/content?id=_jlgl...,The Key to Survival Rests in the Hands of Shad...,1998.0,3.90,345.0,10368.0,Shade's Children (rack),9780064471961 The Key to Survival Rests in the...


In [20]:
books["categories"].value_counts().reset_index()

#we have may catagories which have only 1 counts so now we will do a mapping so that those catagories gets matched with the higest count categories

,categories,count
0,Fiction,2111
1,Juvenile Fiction,390
2,Biography & Autobiography,311
3,History,207
4,Literary Criticism,124
...,...,...
474,Conspiracies,1
475,Brothers and sisters,1
476,Rock musicians,1
477,Community life,1


In [21]:
category_mapping={
    'Fiction':"Fiction",
    'Jvenile Fiction':"Childeren's Fiction",
    'Biography and Autobiography' : "Nonfiction",
    'History':"Nonfiction",
    'Literacy Crirtisism':"Nonfiction",
    'Philosophy': "Nonfiction",
    'Religion': "Nonfiction",
    'Comics & Graphic Novels': "Fiction",
    'Drama': "Fiction",
    'Juvenile Nonfiction': "Children's Nonfiction",
    'Science': "Nonfiction",
    'Poetry':"Fiction"
    }

books["Simple_categories"]= books["categories"].map(category_mapping)
books


,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,Simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NaN
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,NaN
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,NaN
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,NaN
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [22]:
books[~(books["Simple_categories"].isna())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,Simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,Fiction
46,9780007121014,0007121016,Taken at the Flood,Agatha Christie,Fiction,http://books.google.com/books/content?id=3gWlx...,A Few Weeks After Marrying An Attractive Young...,2002.0,3.71,352.0,8852.0,Taken at the Flood,9780007121014 A Few Weeks After Marrying An At...,Fiction
58,9780007149827,0007149824,The Yiddish Policemen's Union,Michael Chabon,Fiction,http://books.google.com/books/content?id=-reD1...,"For sixty years, Jewish refugees and their des...",2007.0,3.70,414.0,51219.0,The Yiddish Policemen's Union: A Novel,"9780007149827 For sixty years, Jewish refugees...",Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5178,9781933648279,1933648279,Night Has a Thousand Eyes,Cornell Woolrich,Fiction,http://books.google.com/books/content?id=3Gk6s...,"""Cornell Woolrich's novels define the essence ...",2007.0,3.77,344.0,680.0,Night Has a Thousand Eyes,"9781933648279 ""Cornell Woolrich's novels defin...",Fiction
5188,9784770028969,4770028962,Coin Locker Babies,村上龍,Fiction,http://books.google.com/books/content?id=87DJw...,Rescued from the lockers in which they were le...,2002.0,3.75,393.0,5560.0,Coin Locker Babies,9784770028969 Rescued from the lockers in whic...,Fiction
5189,9788122200850,8122200850,"Cry, the Peacock",Anita Desai,Fiction,http://books.google.com/books/content?id=_QKwV...,This book is the story of a young girl obsesse...,1980.0,3.22,218.0,134.0,"Cry, the Peacock",9788122200850 This book is the story of a youn...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


***Zero-shot text classification*** is the ability of a model to **classify text into categories without being explicitly trained on those categories**. Instead of needing labeled training data for each possible class, the model understands the relationship between the text and class labels using natural language understanding.

Example of Zero-Shot Text Classification
Imagine you have a sentence:\
📝 "The stock market crashed today, leading to economic concerns." \
And you want to classify it into topics like Finance, Politics, Sports, Technology.

A traditional model would need training data with these labels, but a zero-shot model (like GPT, BERT-based models, or LLaMA) can classify the text into "Finance" without explicit training.


**LLMs use pretrained knowledge and prompt engineering to classify text**. \
The process generally follows these steps:

**Text Encoding** – The model understands the input text by converting it into embeddings. \
**Label Understanding** – The model also converts the category labels into embeddings and checks how similar they are to the text. \
**Semantic Matching** – The model compares text embeddings with category embeddings to find the best match. \
**Probability Estimation** – The model assigns a probability score for each label and picks the highest one. \
This works because LLMs have been trained on vast amounts of general knowledge and can infer category relationships even if they haven't explicitly seen them before. 

In [23]:
from transformers import pipeline
fiction_categories=["Fiction","Nonfiction"]
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

Device set to use cpu


The `.loc` method in pandas is used to access a group of rows and columns by labels or a boolean array

In [29]:
sequence = books.loc[books["Simple_categories"]== "Fiction", "description"].reset_index(drop=True)[0]

In [30]:
pipe(sequence, fiction_categories)

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [31]:
import numpy as np
max_index= np.argmax(pipe(sequence, fiction_categories)["scores"])
max_label= pipe(sequence, fiction_categories)["labels"][max_index]
max_label

'Fiction'

In [41]:
def generate_prediction(sequence, categories):
    predictions= pipe(sequence, categories)
    max_index = np.argmax(predictions["scores"])
    max_label = predictions["labels"][max_index]
    return max_label

In [42]:
#using tqdm we can show the progress bar
from tqdm import tqdm
actual_categories =[]
predicted_categories=[]
for i in tqdm(range(0,300)):
    sequence = books.loc[books["Simple_categories"]== "Fiction", "description"].reset_index(drop=True)[i]
    predicted_categories += [generate_prediction(sequence , fiction_categories)]
    actual_categories += ["Fiction"]

100%|██████████| 300/300 [08:00<00:00,  1.60s/it]


In [43]:
#using tqdm we can show the progress bar
from tqdm import tqdm
actual_categories =[]
predicted_categories=[]
for i in tqdm(range(0,300)):
    sequence = books.loc[books["Simple_categories"]== "Nonfiction", "description"].reset_index(drop=True)[i]
    predicted_categories += [generate_prediction(sequence , fiction_categories)]
    actual_categories += ["Nonfiction"]

100%|██████████| 300/300 [08:04<00:00,  1.61s/it]


In [44]:
prediction_df = pd.DataFrame({"actual_categories": actual_categories, "predicted_categories":predicted_categories})
prediction_df

,actual_categories,predicted_categories
0,Nonfiction,Nonfiction
1,Nonfiction,Nonfiction
2,Nonfiction,Nonfiction
3,Nonfiction,Nonfiction
4,Nonfiction,Nonfiction
...,...,...
295,Nonfiction,Nonfiction
296,Nonfiction,Nonfiction
297,Nonfiction,Nonfiction
298,Nonfiction,Nonfiction


In [45]:
prediction_df["correct_prediction"]=(
    np.where(prediction_df["actual_categories"]== prediction_df["predicted_categories"],1,0)
)

In [46]:
prediction_df["correct_prediction"].sum()/len(prediction_df)

0.94

In [47]:
isbns=[]
predicted_categories=[]

missing_categories = books.loc[books["Simple_categories"].isna() , ["isbn13", "description"]].reset_index(drop=True)

In [48]:
for i in tqdm(range(0, len(missing_categories))):
    sequence = missing_categories["description"][i]
    predicted_categories += [generate_prediction(sequence, fiction_categories)]
    isbns += [missing_categories["isbn13"][i]]

100%|██████████| 2279/2279 [55:54<00:00,  1.47s/it]  


In [49]:
missing_predicted_df = pd.DataFrame({"isbn13":isbns, "predicted_categories":predicted_categories})
missing_predicted_df

,isbn13,predicted_categories
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
2274,9788125026600,Nonfiction
2275,9788171565641,Fiction
2276,9788172235222,Fiction
2277,9788173031014,Nonfiction


In [50]:
books = pd.merge(books, missing_predicted_df, on="isbn13", how="left")
books["Simple_categories"] = np.where(books["Simple_categories"].isna(), books["predicted_categories"], books["Simple_categories"])
books = books.drop(columns = ["predicted_categories"])

In [51]:
books[books["categories"].str.lower().isin([
    "romance",
    "science fiction",
    "fantasy",
    "scifi",
    "horror",
    "mystery",
    "thriller",
    "comedy",
    "crime",
    "historical"
])]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,Simple_categories
24,9780006513087,0006513085,Gravity,Tess Gerritsen,Science fiction,http://books.google.com/books/content?id=KI66c...,Emma Watson a research physician has been trai...,2004.0,4.04,342.0,8024.0,Gravity,9780006513087 Emma Watson a research physician...,Nonfiction
475,9780099410355,0099410354,Traitor,Matthew Woodring Stover,Science fiction,http://books.google.com/books/content?id=VbICO...,"From the depths of catastrophe, a glimmer of h...",2002.0,4.00,320.0,6765.0,Traitor,"9780099410355 From the depths of catastrophe, ...",Fiction
478,9780099422341,0099422344,Yeats is Dead!,Joseph O'Connor,Comedy,http://books.google.com/books/content?id=DrE3I...,"In aid of Amnesty International, this is a bri...",2002.0,3.39,298.0,34.0,Yeats is Dead!: A Novel by Fifteen Irish Writers,"9780099422341 In aid of Amnesty International,...",Fiction
491,9780099446729,0099446723,Blackwood Farm,Anne Rice,Horror,http://books.google.com/books/content?id=cIn8T...,"Lestat Is Back, Saviour And Demon, Presiding O...",2003.0,3.86,774.0,26145.0,Blackwood Farm,"9780099446729 Lestat Is Back, Saviour And Demo...",Fiction
1090,9780261102422,0261102427,The Silmarillion,John Ronald Reuel Tolkien,Fantasy,http://books.google.com/books/content?id=22ePu...,Tolkien's Silmarillion is the core work of the...,1999.0,3.91,384.0,253.0,The Silmarillion,9780261102422 Tolkien's Silmarillion is the co...,Fiction
1435,9780340837955,0340837950,Stranger in a Strange Land,Robert A. Heinlein,Science fiction,http://books.google.com/books/content?id=ZQhiP...,"Epic, entertaining, Stranger in a Strange Land...",2005.0,3.92,672.0,563.0,Stranger in a Strange Land,"9780340837955 Epic, entertaining, Stranger in ...",Fiction
1439,9780345251220,0345251229,Visions from Nowhere,William Arrow,Science fiction,NaN,"The first novel in the series, ""Return to the ...",1976.0,3.23,183.0,10.0,Visions from Nowhere,"9780345251220 The first novel in the series, ""...",Fiction
2845,9780575075597,0575075597,Replay,Ken Grimwood,Fantasy,http://books.google.com/books/content?id=9vmNP...,At forty-three Jeff Winston is tired of his lo...,2005.0,4.16,272.0,412.0,Replay,9780575075597 At forty-three Jeff Winston is t...,Fiction
2860,9780590254762,0590254766,"The lion, the witch and the wardrobe",Clive Staples Lewis,Fantasy,NaN,Four English school children enter the magic l...,1995.0,4.21,189.0,860.0,"The lion, the witch and the wardrobe",9780590254762 Four English school children ent...,Nonfiction
3288,9780739423851,0739423851,Wizard's Castle,Diana Wynne Jones,Fantasy,http://books.google.com/books/content?id=hB7hA...,Howl's moving castle - Eldest of three sisters...,2002.0,4.44,376.0,439.0,Wizard's Castle,9780739423851 Howl's moving castle - Eldest of...,Fiction


In [52]:
books.to_csv("books_with_categories.csv", index=False)